In [1]:
from datasets import load_dataset,Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
def evaluateModel(path):
    common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "zh-HK", split="test")
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    common_voice = common_voice.select(range(1000))

    processor = WhisperProcessor.from_pretrained(path)
    model = WhisperForConditionalGeneration.from_pretrained("../fineTune/fineTue/"+path+"/checkpoint-4000").to("cuda")



    def map_to_pred(batch):
        audio = batch["audio"]
        input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
        batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

        with torch.no_grad():
            predicted_ids = model.generate(input_features.to("cuda"))[0]
        transcription = processor.decode(predicted_ids)
        batch["prediction"] = processor.tokenizer._normalize(transcription)
        return batch

    result = common_voice.map(map_to_pred)

    wer = load("wer")
    print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))



In [2]:
whisper_path=["openai/whisper-tiny","openai/whisper-base","openai/whisper-small","openai/whisper-medium","openai/whisper-large","openai/whisper-large-v2","openai/whisper-large-v3"]

In [3]:
evaluateModel(whisper_path[0])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

169.7239536954586


In [4]:
evaluateModel(whisper_path[1])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

76.40249332146037


In [ ]:
evaluateModel(whisper_path[2])

In [ ]:
evaluateModel(whisper_path[3])

In [ ]:
evaluateModel(whisper_path[4])

In [ ]:
evaluateModel(whisper_path[5])

In [ ]:
evaluateModel(whisper_path[6])